In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

import yaml

In [3]:
mlp_config_path = huggingface_hub.hf_hub_download(
    "jefsnacker/surname_generator",
    "torch_mlp_config.yaml")

mlp_weights_path = huggingface_hub.hf_hub_download(
    "jefsnacker/surname_generator",
    "mlp_weights.pt")

wavenet_config_path = huggingface_hub.hf_hub_download(
    "jefsnacker/surname_generator",
    "wavenet_config.yaml")

wavenet_weights_path = huggingface_hub.hf_hub_download(
    "jefsnacker/surname_generator",
    "wavenet_weights.pt")

Downloading:   0%|          | 0.00/314 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.70M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/313 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/406k [00:00<?, ?B/s]

In [4]:
with open(mlp_config_path, 'r') as file:
    mlp_config = yaml.safe_load(file)

with open(wavenet_config_path, 'r') as file:
    wavenet_config = yaml.safe_load(file)
    

In [5]:
class MLP(nn.Module):
    def __init__(self, num_char, hidden_nodes, embeddings, window, num_layers):   
        super(MLP, self).__init__()
        
        self.window = window
        self.hidden_nodes = hidden_nodes
        self.embeddings = embeddings
        
        self.C = nn.Parameter(torch.randn((num_char, embeddings)) * 0.1, requires_grad=True)
        
        self.first = nn.Linear(embeddings*window, hidden_nodes)

        self.layers = nn.Sequential()
        for i in range(num_layers):
            self.layers = self.layers.extend(nn.Sequential(
                nn.Linear(hidden_nodes, hidden_nodes, bias=False),
                nn.BatchNorm1d(hidden_nodes),
                nn.Tanh()))

        self.final = nn.Linear(hidden_nodes, num_char)
        
    def forward(self, x):
        x = self.C[x]
        x = self.first(x.view(-1, self.window*self.embeddings))
        
        x = self.layers(x)

        x = self.final(x)
        return x
    
    def sample_char(self, x):
        logits = self(x)
        probs = F.softmax(logits, dim=1)
        return torch.multinomial(probs, num_samples=1).item()
     
mlp = MLP(mlp_config['num_char'], 
          mlp_config['hidden_nodes'], 
          mlp_config['embeddings'], 
          mlp_config['window'], 
          mlp_config['num_layers'])

mlp.load_state_dict(torch.load(mlp_weights_path))
mlp.eval()


MLP(
  (first): Linear(in_features=30, out_features=200, bias=True)
  (layers): Sequential(
    (0): Linear(in_features=200, out_features=200, bias=False)
    (1): BatchNorm1d(200, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): Tanh()
    (3): Linear(in_features=200, out_features=200, bias=False)
    (4): BatchNorm1d(200, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): Tanh()
    (6): Linear(in_features=200, out_features=200, bias=False)
    (7): BatchNorm1d(200, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (8): Tanh()
    (9): Linear(in_features=200, out_features=200, bias=False)
    (10): BatchNorm1d(200, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (11): Tanh()
    (12): Linear(in_features=200, out_features=200, bias=False)
    (13): BatchNorm1d(200, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (14): Tanh()
    (15): Linear(in_features=200, out_features=200, bias=F

In [6]:
class WaveNet(nn.Module):
    def __init__(self, num_char, hidden_nodes, embeddings, window, num_layers):   
        super(WaveNet, self).__init__()
        
        self.window = window
        self.hidden_nodes = hidden_nodes
        self.embeddings = embeddings        
        
        self.layers = nn.Sequential(
            nn.Embedding(num_char, embeddings)
        )
        
        for i in range(num_layers):
            if i == 0:
                nodes = window
            else:
                nodes = hidden_nodes
                
            self.layers = self.layers.extend(nn.Sequential(
                nn.Conv1d(nodes, hidden_nodes, kernel_size=2, stride=1, bias=False),
                nn.BatchNorm1d(hidden_nodes),
                nn.Tanh()))
            
        self.layers = self.layers.extend(nn.Sequential(
            nn.Flatten(),
            nn.Linear(hidden_nodes*(embeddings-num_layers), num_char)
        ))
        
    def forward(self, x):
        return self.layers(x)
    
    def sample_char(self, x):
        logits = self(x)
        probs = F.softmax(logits, dim=1)
        return torch.multinomial(probs, num_samples=1).item()
    
wavenet = WaveNet(wavenet_config['num_char'], 
                  wavenet_config['hidden_nodes'], 
                  wavenet_config['embeddings'], 
                  wavenet_config['window'], 
                  wavenet_config['num_layers'])
wavenet.load_state_dict(torch.load(wavenet_weights_path))
wavenet.eval()

WaveNet(
  (layers): Sequential(
    (0): Embedding(30, 10)
    (1): Conv1d(8, 100, kernel_size=(2,), stride=(1,), bias=False)
    (2): BatchNorm1d(100, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (3): Tanh()
    (4): Conv1d(100, 100, kernel_size=(2,), stride=(1,), bias=False)
    (5): BatchNorm1d(100, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): Tanh()
    (7): Conv1d(100, 100, kernel_size=(2,), stride=(1,), bias=False)
    (8): BatchNorm1d(100, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (9): Tanh()
    (10): Conv1d(100, 100, kernel_size=(2,), stride=(1,), bias=False)
    (11): BatchNorm1d(100, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (12): Tanh()
    (13): Conv1d(100, 100, kernel_size=(2,), stride=(1,), bias=False)
    (14): BatchNorm1d(100, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (15): Tanh()
    (16): Flatten(start_dim=1, end_dim=-1)
    (17): Linear(i

In [7]:
next(mlp.parameters()).device, next(wavenet.parameters()).device

(device(type='cpu'), device(type='cpu'))

In [8]:
import gradio as gr

def generate_names(name_start, number_of_names, model):
    if model == "MLP":
        stoi = mlp_config['stoi']
        window = mlp_config['window']
    elif model == "WaveNet":
        stoi = wavenet_config['stoi']
        window = wavenet_config['window']
    itos = {s:i for i,s in stoi.items()}

    names = ""
    for _ in range((int)(number_of_names)):
    
        # Initialize name with user input
        name = ""
        context = [0] * window
        for c in name_start.lower():
            name += c
            context = context[1:] + [stoi[c]]

        # Run inference to finish off the name
        while True:
            x = torch.tensor(context).view(1, -1)
            if model == "MLP":
                ix = mlp.sample_char(x)
            elif model == "WaveNet":
                ix = wavenet.sample_char(x)
                
            context = context[1:] + [ix]
            name += itos[ix]
                
            if ix == 0:
                break
            
        names += name + "\n"
        
    return names

demo = gr.Interface(
    fn=generate_names,
    inputs=[
        gr.Textbox(placeholder="Start name with..."),
        gr.Number(value=5),
        gr.Dropdown(["MLP", "WaveNet"], value="WaveNet"),
    ],
    outputs="text",
)
demo.launch()

Running on local URL:  http://127.0.0.1:7866

To create a public link, set `share=True` in `launch()`.


[W NNPACK.cpp:53] Could not initialize NNPACK! Reason: Unsupported hardware.
